In [1]:
!pip install torch torchvision timm ultralytics gdown
!pip install torchinfo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [21]:
# ✅ Step 1: Make sure folder exists
import os
os.makedirs("model_weights", exist_ok=True)

# ✅ Step 2: Download all 3 models using --fuzzy
import gdown

# InceptionV3
gdown.download('https://drive.google.com/file/d/1XVpkjBwATr8rOJ0DQ-bg-H0lFH7D7Xrz/view?usp=drive_link',
               'model_weights/best_inceptionv3.pth', fuzzy=True)

# ViT
gdown.download('https://drive.google.com/file/d/1f7ZXymjA3lFo4A_4C2LKPI--79djRZKE/view?usp=drive_link',
               'model_weights/best_vit.pth', fuzzy=True)

# YOLO
gdown.download('https://drive.google.com/file/d/13PNaxtw7WqO_DYvFyG9e91VRbVyFMfFl/view?usp=drive_link',
               'model_weights/best_yolo.pt', fuzzy=True)


Downloading...
From (original): https://drive.google.com/uc?id=1XVpkjBwATr8rOJ0DQ-bg-H0lFH7D7Xrz
From (redirected): https://drive.google.com/uc?id=1XVpkjBwATr8rOJ0DQ-bg-H0lFH7D7Xrz&confirm=t&uuid=b3f96f8d-a906-45c3-aa07-1e9e748c0d2f
To: /content/yolov5/model_weights/best_inceptionv3.pth
100%|██████████| 105M/105M [00:00<00:00, 186MB/s] 
Downloading...
From (original): https://drive.google.com/uc?id=1f7ZXymjA3lFo4A_4C2LKPI--79djRZKE
From (redirected): https://drive.google.com/uc?id=1f7ZXymjA3lFo4A_4C2LKPI--79djRZKE&confirm=t&uuid=a77c5281-90f4-4778-9c65-087b40a74d4c
To: /content/yolov5/model_weights/best_vit.pth
100%|██████████| 345M/345M [00:01<00:00, 195MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=13PNaxtw7WqO_DYvFyG9e91VRbVyFMfFl
From (redirected): https://drive.google.com/uc?id=13PNaxtw7WqO_DYvFyG9e91VRbVyFMfFl&confirm=t&uuid=31b21b5d-8105-4ada-bfe4-4329c3d034e8
To: /content/yolov5/model_weights/best_yolo.pt
100%|██████████| 40.5M/40.5M [00:00<00:00, 135MB/s]

'model_weights/best_yolo.pt'

In [3]:
import os
print(os.listdir("model_weights"))


['best_inceptionv3.pth', 'best_vit.pth', 'best_yolo.pt']


In [4]:
%matplotlib inline

import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
import pandas as pd
import numpy as np
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler


import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torch.utils.data import Subset
from torchvision import datasets, transforms
import sys
import time
from PIL import Image

In [5]:
!pip install torchinfo


In [6]:
import torch
import gc

gc.collect()
torch.cuda.empty_cache()

In [7]:
# For reproduce the result
torch.manual_seed(1220)
random.seed(1220)
np.random.seed(1220)

In [8]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)
print("You are using PyTorch version: %s" % torch.__version__)



You are using device: cpu
You are using PyTorch version: 2.6.0+cu124


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
!apt install unrar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [11]:
!unrar x "/content/drive/MyDrive/datasets.rar" "/content/datasets/"

Streaming output truncated to the last 5000 lines.
Extracting  /content/datasets/crack_classification/original/column/Vertical/cropped_129_jpg.rf.2220c7f7973cba820135c026acb9f5b0.jpg      98%  OK 
Extracting  /content/datasets/crack_classification/original/column/Vertical/cropped_CLD_000_jpg.rf.057e703fcfe61dacfc181b0bafab3e20.jpg      98%  OK 
Extracting  /content/datasets/crack_classification/original/column/Vertical/cropped_CLD_005_jpg.rf.c492ceedeb9d9e528baa2358b70449a5.jpg      98%  OK 
Extracting  /content/datasets/crack_classification/original/column/Vertical/cropped_CLD_010_jpg.rf.af3b508fc756f769ccb6ffe1ec878e22.jpg      98%  OK 
Extracting  /content/datasets/crack_classification/original/column/Vertical/cropped_CLD_011_jpg.rf.c1750f64662b3bc3b987d88525e8080f.jpg      98%  OK 
Extracting  /content/datasets/crack_classification/original/column/Vertical/cropped_CLD_014_jpg.rf.b7ace753fe607763f959f9731084bbe4.jpg      98%  OK 

Using Inception V3 for Damage Classification

In [12]:
from torch.utils.data import WeightedRandomSampler
from collections import Counter

# Load dataset
path_dataset = "/content/datasets/damage_classification_forTrain/column_damage"


tta_transforms = [
    transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]),
    transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.RandomHorizontalFlip(p=1.0),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]),
    transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
]

def load_dataset(path_dataset):
    # Data transformations for training
    transform_train = transforms.Compose([
        transforms.Resize([299, 299]),
        transforms.RandomCrop(299, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
        transforms.RandomAffine(degrees=10, translate=(0.1,0.1), scale =(0.9,1.1)),
        transforms.ToTensor(),
        #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                     std=[0.229, 0.224, 0.225])

    ])

    ########################I added extra data augmentation for my training data

    # Data transformations for testing
    transform_test = transforms.Compose([
        transforms.Resize([299, 299]) ,
        transforms.ToTensor(),
        #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                     std=[0.229, 0.224, 0.225])
         #apply only basic transformation to test data, no randomness
    ])

    # Create a non-augmented(no need for normalize) transform for visualization
    transform_noaug = transforms.Compose([
        transforms.Resize([299, 299]),
        transforms.ToTensor()
    ])


    full_dataset = datasets.ImageFolder(root=path_dataset, transform=None)  # No transform yet
    indices = torch.randperm(len(full_dataset))  # Fixed seed ensures reproducibility
    train_size = int(0.8 * len(full_dataset))
    train_indices, val_indices = indices[:train_size], indices[train_size:]

    trainset = Subset(
    datasets.ImageFolder(root=path_dataset, transform = transform_train),
    train_indices
    )
    valset = Subset(
    datasets.ImageFolder(root=path_dataset, transform = transform_test),
    val_indices
    )

    # ----- Create WeightedRandomSampler -----



    # also for visualization
    full_dataset_noaug = torchvision.datasets.ImageFolder(
        root=path_dataset,
        transform=transform_noaug
    )


    # Create data loaders
    trainloader = DataLoader(
        trainset, batch_size=32, shuffle=True, num_workers=2  #pytorch tutorial
    )

    valloader = DataLoader(
        valset, batch_size=32, shuffle=False, num_workers=2 #pytorch tutorial
    )


    # Map folder names to class names
    class_map = {i: full_dataset.classes[i] for i in range(len(full_dataset.classes))}
    print(f"Dataset classes: {class_map}")
    print("Using classes from dataset folder structure:", full_dataset.classes)


    return trainloader, valloader, full_dataset_noaug

# Define a composite transform for data augmentation for "visualizations" only, you can try to implement it for training
train_transformer_agumentation = transforms.Compose([
    transforms.ToPILImage(),  # Convert tensor to PIL Image (the format for transforms)
    transforms.RandomChoice([
        transforms.RandomCrop(299, padding=4, padding_mode='reflect'),
        transforms.RandomRotation(90),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomAffine(degrees=0, shear=(0, 0, 0, 45)),
        transforms.RandomPerspective(distortion_scale=0.5, fill=0),
        transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
    ]),

    transforms.ToTensor(),  # Convert back to tensor (the format for models)
])
# Transform for inference
train_transformer_inference = transforms.Compose([
    transforms.Resize([299, 299]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])



print('Classes found:', sorted(os.listdir(path_dataset)))
trainloader, valloader, dataset_noaug = load_dataset(path_dataset)

Classes found: ['Class A', 'Class B', 'Class C']
Dataset classes: {0: 'Class A', 1: 'Class B', 2: 'Class C'}
Using classes from dataset folder structure: ['Class A', 'Class B', 'Class C']


In [13]:
import math

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F

from torchinfo import summary
from torchvision.models import inception_v3, Inception_V3_Weights


class DamageClassifier(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()
        weights = Inception_V3_Weights.DEFAULT
        base = inception_v3(weights=weights, aux_logits=True)  # aux_logits=True is required
        num_ftrs = base.fc.in_features
        base.fc = nn.Identity()
        self.backbone = base

        self.fc1 = nn.Linear(num_ftrs, 512)
        self.dropout = nn.Dropout(0.3)
        self.cls_head = nn.Linear(512, num_classes)

    def forward(self, x):
        output = self.backbone(x)
        if isinstance(output, tuple):  # Handle auxiliary logits
            output = output[0]
        x = torch.relu(self.fc1(output))
        x = self.dropout(x)
        return self.cls_head(x)

model_ = DamageClassifier(num_classes=3).to(device)
print(model_)
summary(model_, input_size=(8, 3, 299, 299))


Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 189MB/s] 


DamageClassifier(
  (backbone): Inception3(
    (Conv2d_1a_3x3): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2a_3x3): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2b_3x3): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (Conv2d_3b_1x1): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_4a_3x3

Layer (type:depth-idx)                   Output Shape              Param #
DamageClassifier                         [8, 3]                    --
├─Inception3: 1-1                        [8, 2048]                 3,326,696
│    └─BasicConv2d: 2-1                  [8, 32, 149, 149]         --
│    │    └─Conv2d: 3-1                  [8, 32, 149, 149]         864
│    │    └─BatchNorm2d: 3-2             [8, 32, 149, 149]         64
│    └─BasicConv2d: 2-2                  [8, 32, 147, 147]         --
│    │    └─Conv2d: 3-3                  [8, 32, 147, 147]         9,216
│    │    └─BatchNorm2d: 3-4             [8, 32, 147, 147]         64
│    └─BasicConv2d: 2-3                  [8, 64, 147, 147]         --
│    │    └─Conv2d: 3-5                  [8, 64, 147, 147]         18,432
│    │    └─BatchNorm2d: 3-6             [8, 64, 147, 147]         128
│    └─MaxPool2d: 2-4                    [8, 64, 73, 73]           --
│    └─BasicConv2d: 2-5                  [8, 80, 73, 73]           --

In [14]:
import torch.nn.functional as F
from PIL import Image

def predict_tta(model, image_path, tta_transforms):
    model.eval()
    image = Image.open(image_path).convert('RGB')

    all_preds = []

    with torch.no_grad():
        for transform in tta_transforms:
            img_tensor = transform(image).unsqueeze(0).to(device)
            output = model(img_tensor)
            prob = F.softmax(output, dim=1)
            all_preds.append(prob)

    avg_output = torch.stack(all_preds).mean(dim=0)  # shape: [1, num_classes]
    predicted_class = torch.argmax(avg_output, dim=1).item()
    return predicted_class



In [15]:
from tqdm import tqdm
import os
from PIL import Image
import pandas as pd
import torch

# Paths
test_folder_path = "/content/datasets/test_data/column"
model_path = "model_weights/best_inceptionv3.pth"
file_extension = ".jpg"
csv_file_path = "/content/datasets/final_column_test.csv"  # new output file

# Class mapping: model output index → competition class label
idx_to_class = {0: 18, 1: 19, 2: 20}  # A → 18, B → 19, C → 20

# Load model
model_.load_state_dict(torch.load(model_path, map_location=device))
model_.to(device)
model_.eval()

# Get list of test image IDs
image_files = sorted([f for f in os.listdir(test_folder_path) if f.endswith(file_extension)])
image_ids = [os.path.splitext(f)[0] for f in image_files]

# Create new CSV DataFrame
df = pd.DataFrame({'id': image_ids, 'class': ''})

# Predict for each image using TTA
for idx, img_id in tqdm(enumerate(df['id']), total=len(df), desc="Generating TTA Predictions"):
    try:
        img_path = os.path.join(test_folder_path, img_id + file_extension)
        predicted_idx = predict_tta(model_, img_path, tta_transforms)
        df.at[idx, 'class'] = idx_to_class[predicted_idx]  # map to 18/19/20
    except Exception as e:
        print(f"❌ Error with image {img_id}: {e}")

# Save CSV
df.to_csv(csv_file_path, index=False)
print(f"✅ Submission CSV saved at: {csv_file_path}")


Generating TTA Predictions: 100%|██████████| 50/50 [00:18<00:00,  2.63it/s]

✅ Submission CSV saved at: /content/datasets/final_column_test.csv


In [16]:
import pandas as pd

# Load your CSV (replace with your actual file path)
df = pd.read_csv("/content/datasets/final_column_test.csv")

# Ensure ID is treated as an integer
df['id'] = df['id'].astype(int)

# Sort by ID
df_sorted = df.sort_values(by='id').reset_index(drop=True)

# Save to a new CSV
df_sorted.to_csv("/content/datasets/sorted_final_column_test.csv", index=False)

print("✅ Sorted CSV saved as 'sorted_submission.csv'")


✅ Sorted CSV saved as 'sorted_submission.csv'


In [17]:
from google.colab import files
files.download("/content/datasets/sorted_final_column_test.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

YOLO: Damage Detection

In [18]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17488, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 17488 (delta 0), reused 0 (delta 0), pack-reused 17486 (from 2)
Receiving objects: 100% (17488/17488), 16.54 MiB | 30.56 MiB/s, done.
Resolving deltas: 100% (11988/11988), done.
/content/yolov5


In [22]:
from ultralytics import YOLO

# Load your trained model
model = YOLO("model_weights/best_yolo.pt")
# Run detection on your test set
results = model.predict(
    source="/content/datasets/test_data/column",
    conf=0.25,
    save=False,
    save_txt=True,
    save_conf=True
)





image 1/50 /content/datasets/test_data/column/1.jpg: 640x192 1 crack, 1 spalling, 349.9ms
image 2/50 /content/datasets/test_data/column/10.jpg: 640x256 1 crack, 283.7ms
image 3/50 /content/datasets/test_data/column/11.jpg: 640x160 2 cracks, 177.7ms
image 4/50 /content/datasets/test_data/column/12.jpg: 640x160 1 crack, 2 spallings, 147.8ms
image 5/50 /content/datasets/test_data/column/13.jpg: 640x128 3 spallings, 156.2ms
image 6/50 /content/datasets/test_data/column/14.jpg: 640x416 1 rebar, 1 spalling, 358.6ms
image 7/50 /content/datasets/test_data/column/15.jpg: 640x352 3 cracks, 1 spalling, 325.7ms
image 8/50 /content/datasets/test_data/column/16.jpg: 640x320 2 cracks, 2 spallings, 273.3ms
image 9/50 /content/datasets/test_data/column/17.jpg: 640x416 1 crack, 1 spalling, 307.5ms
image 10/50 /content/datasets/test_data/column/18.jpg: 640x160 3 spallings, 140.0ms
image 11/50 /content/datasets/test_data/column/19.jpg: 448x640 4 cracks, 326.0ms
image 12/50 /content/datasets/test_data/col

In [23]:
import os

yolo_dir = "runs/detect/predict/labels"
image_ids = [f.replace(".txt", "") for f in os.listdir(yolo_dir)]

detections = {}
for file in os.listdir(yolo_dir):
    with open(os.path.join(yolo_dir, file)) as f:
        rows = [line.strip().split() for line in f]
    class_ids = [int(row[0]) for row in rows]
    detections[file.replace(".txt", "")] = class_ids  # dict[image_id] = [0, 1, 2]


In [25]:
import pandas as pd

# Load class predictions from CSV
class_df = pd.read_csv("/content/datasets/sorted_final_column_test.csv")

# Create a dictionary for fast lookup
damage_classes = dict(zip(class_df["id"].astype(str), class_df["class"]))


Getting cropped images from yolo boxes

In [26]:
def get_yolo_boxes(img_id, target_class, label_dir="runs/detect/predict/labels"):
    boxes = []
    label_path = os.path.join(label_dir, img_id + ".txt")
    if not os.path.exists(label_path):
        return boxes

    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            cls = int(parts[0])
            if cls == target_class:
                # YOLO format: class x_center y_center width height
                boxes.append([float(x) for x in parts[1:5]])
    return boxes


In [27]:
from PIL import Image

def crop_image(img_path, yolo_box):
    # yolo_box = [x_center, y_center, width, height] (all normalized)
    image = Image.open(img_path).convert("RGB")
    W, H = image.size

    x_c, y_c, w, h = [x * W if i % 2 == 0 else x * H for i, x in enumerate(yolo_box)]
    x1 = int(max(x_c - w / 2, 0))
    y1 = int(max(y_c - h / 2, 0))
    x2 = int(min(x_c + w / 2, W))
    y2 = int(min(y_c + h / 2, H))

    return image.crop((x1, y1, x2, y2))

In [28]:
from torchvision.datasets import ImageFolder
from PIL import Image

class ImageFolderWithHough(ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        image = Image.open(path).convert("RGB")

        # Extract visual and Hough features
        hough_feats = extract_hough_features_advanced(image)
        image_tensor = self.transform(image)

        return image_tensor, hough_feats, target


In [29]:
import cv2
import numpy as np
from scipy.stats import entropy
from itertools import combinations

def extract_hough_features_advanced(pil_image):
    image = np.array(pil_image.convert("RGB"))
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 50, 150)

    # --- 1. Standard Hough Line Transform for angle features ---
    lines = cv2.HoughLines(edges, 1, np.pi / 180, threshold=50)
    theta_list = []

    horiz = vert = diag_45 = diag_135 = 0

    if lines is not None:
        for rho, theta in lines[:, 0]:
            deg = np.degrees(theta)
            theta_list.append(deg)
            if abs(deg - 0) < 10 or abs(deg - 180) < 10:
                horiz += 1
            elif abs(deg - 90) < 10:
                vert += 1
            elif abs(deg - 45) < 10:
                diag_45 += 1
            elif abs(deg - 135) < 10:
                diag_135 += 1

    # Histogram & entropy of angles
    hist, _ = np.histogram(theta_list, bins=np.linspace(0, 180, 19))  # 10° bins
    angle_entropy = entropy(hist + 1e-6)
    num_peaks = np.sum(hist > 5)

    # --- 2. Probabilistic Hough Line Transform for lengths and intersections ---
    linesP = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=30, minLineLength=20, maxLineGap=5)
    avg_length = 0
    intersection_count = 0

    if linesP is not None and len(linesP) > 1:
        lengths = []
        for x1, y1, x2, y2 in linesP[:, 0]:
            lengths.append(np.hypot(x2 - x1, y2 - y1))
        avg_length = np.mean(lengths)

        # Count pairwise line intersections
        def ccw(A, B, C):
            return (C[1]-A[1]) * (B[0]-A[0]) > (B[1]-A[1]) * (C[0]-A[0])

        def lines_intersect(A, B, C, D):
            return ccw(A, C, D) != ccw(B, C, D) and ccw(A, B, C) != ccw(A, B, D)

        for (x1a, y1a, x2a, y2a), (x1b, y1b, x2b, y2b) in combinations(linesP[:, 0], 2):
            if lines_intersect((x1a, y1a), (x2a, y2a), (x1b, y1b), (x2b, y2b)):
                intersection_count += 1

    # --- 3. Dominance Ratio ---
    total_dirs = horiz + vert + diag_45 + diag_135 + 1e-6
    dominance_ratio = max(horiz, vert, diag_45, diag_135) / total_dirs

    # Final feature vector
    feature_vector = np.array([
        horiz,
        vert,
        diag_45,
        diag_135,
        num_peaks,
        angle_entropy,
        avg_length,
        intersection_count,
        dominance_ratio
    ], dtype=np.float32)

    return feature_vector


In [30]:
import os
import shutil
import random

src_dir = "/content/datasets/crack_classification/original/column"
target_dir = "/content/datasets/crack_classification/split/column"

# Define train/val folders
train_dir = os.path.join(target_dir, "train")
val_dir = os.path.join(target_dir, "val")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# For each class folder
for class_name in os.listdir(src_dir):
    class_path = os.path.join(src_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    images = os.listdir(class_path)
    random.shuffle(images)

    split_idx = int(0.8 * len(images))
    train_imgs = images[:split_idx]
    val_imgs = images[split_idx:]

    # Make subfolders
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)

    # Move files
    for img in train_imgs:
        shutil.copy(os.path.join(class_path, img), os.path.join(train_dir, class_name, img))
    for img in val_imgs:
        shutil.copy(os.path.join(class_path, img), os.path.join(val_dir, class_name, img))

print("✅ Dataset split complete.")


✅ Dataset split complete.


In [31]:
import cv2
import numpy as np

def extract_hough_features(pil_image):
    image = np.array(pil_image.convert("RGB"))
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    lines = cv2.HoughLines(edges, 1, np.pi / 180, threshold=50)

    # Initialize direction counters
    horiz = vert = diag_45 = diag_135 = 0
    theta_list = []

    if lines is not None:
        for rho, theta in lines[:, 0]:
            deg = np.degrees(theta)
            theta_list.append(deg)
            if abs(deg - 0) < 10 or abs(deg - 180) < 10:
                horiz += 1
            elif abs(deg - 90) < 10:
                vert += 1
            elif abs(deg - 45) < 10:
                diag_45 += 1
            elif abs(deg - 135) < 10:
                diag_135 += 1

    # Histogram-based peak count
    hist, _ = np.histogram(theta_list, bins=np.arange(0, 181, 5))
    num_peaks = np.sum(hist > 5)

    return np.array([horiz, vert, diag_45, diag_135, num_peaks], dtype=np.float32)


In [32]:
import torch.nn as nn

class ViTWithHough(nn.Module):
    def __init__(self, vit_model, num_classes, hough_dim=9):
        super(ViTWithHough, self).__init__()
        self.vit = vit_model
        vit_out_dim = self.vit.heads[0].in_features
        self.vit.heads = nn.Identity()  # Remove old head
        self.classifier = nn.Sequential(
            nn.Linear(vit_out_dim + hough_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, images, hough_feats):
        vit_feats = self.vit(images)
        combined = torch.cat((vit_feats, hough_feats), dim=1)
        return self.classifier(combined)


In [33]:
# Load base ViT
vit = models.vit_b_16(weights=models.ViT_B_16_Weights.DEFAULT)

# Wrap it with your custom Hough-aware classifier
model = ViTWithHough(vit_model=vit, num_classes=7, hough_dim =9).to(device)




Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:02<00:00, 117MB/s]


In [34]:
# Load ViT model

# 1. Load base ViT (do NOT modify heads)
vit = models.vit_b_16(weights=models.ViT_B_16_Weights.DEFAULT)

# 2. Wrap with your Hough-aware classifier
model = ViTWithHough(vit_model=vit, num_classes=7).to(device)

# 3. Load the full trained model (ViT + Hough)
model.load_state_dict(torch.load("model_weights/best_vit.pth", map_location=device))
model.eval()

# Image transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Crack class names (must match training)
class_names = ['Diagonal', 'Horizontal', 'Horizontal_large', 'Vertical', 'Vertical_large', 'Web', 'X-shape']



def predict_vit(model, image, transform, class_names):
    # Transform the image
    image_tensor = transform(image).unsqueeze(0).to(device)

    # Extract Hough features (advanced version!)
    hough_feats = extract_hough_features_advanced(image)  # Must return 9 features
    hough_tensor = torch.tensor(hough_feats, dtype=torch.float32).unsqueeze(0).to(device)

    # Predict
    model.eval()
    with torch.no_grad():
        output = model(image_tensor, hough_tensor)
        pred = torch.argmax(output, dim=1).item()

    return class_names[pred]


Label Mapping

In [35]:
criteria_map = {
    "column": {
        18: [0, 3, 4, 6, 8],
        19: [5, 7],
        20: [1, 9, 10]
    },
    # also for wall, beam
}

In [36]:
def map_crack_label_to_criteria(label_name, damage_label, structure_type="column"):
    name = label_name.lower()

    if structure_type == "column":
        if "diagonal" in name:
            return 4 if damage_label == 18 else 5 if damage_label == 19 else None
        elif "horizontal_large" in name or "horizontal" in name:
            return 8 if damage_label == 18 else 9 if damage_label == 20 else None
        elif "vertical_large" in name or "vertical" in name:
            return 6 if damage_label == 18 else 7 if damage_label == 19 else None
        elif "x" in name:
            return 3 if damage_label == 18 else 10 if damage_label == 10 else None
        elif "web" in name:
            return 10 if damage_label == 20 else None
        else:
            return None
    else:
        return None


In [37]:
rows = []
structure = "column"
image_folder = "/content/datasets/test_data/column"

for img_id in image_ids:
    cls = damage_classes[img_id]  # 18/19/20
    detected_classes = detections.get(img_id, [])
    criteria = []

    if 0 in detected_classes:
        criteria.append(0)  # Exposed rebar → 0

    if 1 in detected_classes:
        crack_boxes = get_yolo_boxes(img_id, 1)  # class 1 = crack
        img_path = os.path.join(image_folder, img_id + ".jpg")
        for box in crack_boxes:
            crop = crop_image(img_path, box)
            #label = predict_vit(vit, crop, transform, class_names)
            label = predict_vit(model, crop, transform, class_names)

            mapped = map_crack_label_to_criteria(label, cls, structure)
            if mapped is not None:
                criteria.append(mapped)

    # Ignore class 2 (spalling) for columns

    # Filter by allowed criteria for this damage class
    allowed = criteria_map[structure][cls]
    filtered = list(set(criteria) & set(allowed))

    row = [img_id, cls] + sorted(filtered)
    rows.append(row)



In [42]:
# Step 1: Rebuild rows with merged class + criteria
submission_rows = []

for row in rows:
    img_id = int(row[0])  # make sure sorting works as integers
    labels = row[1:]      # class + criteria
    label_str = ",".join(str(x) for x in labels)
    submission_rows.append([img_id, label_str])

# Step 2: Convert to DataFrame and sort by id
submission_df = pd.DataFrame(submission_rows, columns=["id", "class"])
submission_df = submission_df.sort_values(by="id").reset_index(drop=True)

# Step 3: Save in correct format
submission_df.to_csv("final_submission_column.csv", index=False, header=True)
df = pd.read_csv("final_submission_column.csv")
df = df.rename(columns={"id": "ID"})
df.to_csv("final.csv", index=False)




In [43]:
from google.colab import files
files.download("final.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>